<a href="https://colab.research.google.com/github/gabrielcostapassos/Python-Data-Science/blob/master/Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1 - Configurando o Spark no Python**

Etapa responsável por realizar o Download e configuração do Apache Spark.


In [ ]:
#Instacao e configuracao do Java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Download do Apache Spark
!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
#Descompatacao do Apache Spark
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
#Instalacao do Findspark
!pip install -q findspark

**2 - Configurando o Ambiente**

Etapa responsável por configurar o ambiente do Spark


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

**3 - Configurando a sessão do Spark**

Etapa responsável por inicializar a sessão do Spark


In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

**4 - Validando a Configuração e Versão do Spark**

Etapa de verificação


In [ ]:
import pyspark
print(pyspark.__version__)

3.0.1


**5 - Download dos Dados do Portal da Transparência**

Etapa responsável por realizar o Download das informações de Despesas providas através da API: http://www.portaltransparencia.gov.br/api-de-dados/despesas/recursos-recebidos?pagina=1


In [ ]:
!curl -X GET --header 'Accept: application/json' --header 'chave-api-dados: efe6aa1f980a2c05fdedff7464cac567' 'http://www.portaltransparencia.gov.br/api-de-dados/despesas/recursos-recebidos?mesAnoInicio=01%2F2020&mesAnoFim=08%2F2020&pagina=1' > consulta_recursos_recebidos.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6667    0  6667    0     0   2439      0 --:--:--  0:00:02 --:--:--  2438


**6 - Carregando os dados no Spark**

Etapa paga a carga de dados no Spark.


In [ ]:
sc = spark.sparkContext
#Realiza a leitura da nossa consulta realizada na API do Governo Federal sobre as Viagens
path = "/content/consulta_recursos_recebidos.json"
#Declara dataframe df com o conteudo da api que estava no JSON
df = spark.read.json(path)

**7 - Explorando os dados que foram obtidos**

Etapa de exploração dos dados.


In [ ]:
#Imprime a estrutura de dados do nosso dataframe
df.printSchema()
#Imprime uma amosta de dados ( para especificar as linhas basta colocar a quantidade entre ())
df.show()

root
 |-- anoMes: long (nullable = true)
 |-- codigoFormatado: string (nullable = true)
 |-- codigoOrgao: string (nullable = true)
 |-- codigoOrgaoSuperior: string (nullable = true)
 |-- codigoUG: string (nullable = true)
 |-- municipioPessoa: string (nullable = true)
 |-- nomeOrgao: string (nullable = true)
 |-- nomeOrgaoSuperior: string (nullable = true)
 |-- nomePessoa: string (nullable = true)
 |-- nomeUG: string (nullable = true)
 |-- siglaUFPessoa: string (nullable = true)
 |-- tipoPessoa: string (nullable = true)
 |-- valor: double (nullable = true)

+------+------------------+-----------+-------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+----------+
|anoMes|   codigoFormatado|codigoOrgao|codigoOrgaoSuperior|codigoUG|     municipioPessoa|           nomeOrgao|   nomeOrgaoSuperior|          nomePessoa|              nomeUG|siglaUFPessoa|          tipoPessoa|     val

**8 - Tratando e transformando os dados**


In [ ]:
dfPrepared = df.selectExpr("municipioPessoa","anoMes as AnoMes","nomePessoa","tipoPessoa","valor")
dfPrepared.show()

+--------------------+------+--------------------+--------------------+----------+
|     municipioPessoa|AnoMes|          nomePessoa|          tipoPessoa|     valor|
+--------------------+------+--------------------+--------------------+----------+
|      ROSÁRIO DO SUL|202004|MUNICIPIO DE ROSA...|Administração Púb...|  44571.43|
|           PETROLINA|202008|GUARDSECURE SEGUR...|Entidades Empresa...|  35244.83|
|      RIO DE JANEIRO|202003|JOAO LUIZ MAURITY...|       Pessoa Física|    2400.0|
|          ROCA SALES|202002|       DIESSY KIPPER|       Pessoa Física|     800.0|
|      BELO HORIZONTE|202008|CARVALHO MATERIAI...|Entidades Empresa...|    1482.0|
|         FARROUPILHA|202007|ALPHA HIGIENE E L...|Entidades Empresa...|    1155.0|
|        JUIZ DE FORA|202004|ANA LETICIA OLIVE...|       Pessoa Física|     400.0|
|BOM JESUS DO ARAG...|202003|NAILSON NASCIMENT...|       Pessoa Física|     973.5|
|            BRASÍLIA|202003|LAYANE PEREIRA DE...|       Pessoa Física|     450.0|
|   

**9 - Analise das informações e resultados obtidos**

Etapa onde são realizadas as analises utilizando linguagem SQL para por exemplo descobrir quem mais gastou no ano vigente.

In [ ]:
#Declara tabela temporaria no SparkSQL
dfPrepared.createOrReplaceTempView("dfPrepared")

In [ ]:
topTravels = spark.sql("select AnoMes, nomePessoa, valor from dfPrepared where valor > 500000")
topTravels.show()


+------+--------------------+----------+
|AnoMes|          nomePessoa|     valor|
+------+--------------------+----------+
|202004|FUNDO MUNICIPAL D...|1131891.65|
+------+--------------------+----------+

